In [21]:
import re
import numpy as np
# import cv2
# import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from skimage.draw import polygon

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import numpy as np
from scipy.spatial.distance import directed_hausdorff
from skimage.metrics import hausdorff_distance

plt.rcParams.update({'font.size': 8})
plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = ['Times New Roman']

def parse_coordinates(file_path):
    model_outputs = []
    ground_truths = []

    with open(file_path, 'r') as file:
        for line in file:
            # Identify sections for ground truth and model output
            gt_match = re.search(r'Ground truth:(.*?)(?=Model output:|$)', line)
            model_match = re.search(r'Model output:(.*)', line)

            # Extract coordinates separately for each part
            gt_coords = re.findall(r'\(\s*(\d+)\s*,\s*(\d+)\s*\)', gt_match.group(1)) if gt_match else []
            model_coords = re.findall(r'\(\s*(\d+)\s*,\s*(\d+)\s*\)', model_match.group(1)) if model_match else []

            # Convert to integer tuples
            gt_coords = [(int(x), int(y)) for x, y in gt_coords]
            model_coords = [(int(x), int(y)) for x, y in model_coords]

            # Ensure model output has at least one matching entry
            if not model_coords:
                model_coords = [(-1, -1)] * len(gt_coords)

            ground_truths.append(gt_coords)
            model_outputs.append(model_coords)

    return model_outputs, ground_truths




def convert_to_numpy_array(nested_list, num_points=15):
    num_patients = 1
    num_slices = len(nested_list)
    # Initialize a numpy array with np.nan
    numpy_array = np.full((num_slices, num_points, 2), -1)
    for j in range(num_slices):
        points = nested_list[j]
        if len(points) > 0:
            # Adjust num_points if len(points) is greater
            num_points_adjusted = max(num_points, len(points))
            # Pad the points with -1 if there are fewer than num_points
            padded_points = np.pad(points, ((0, num_points_adjusted - len(points)), (0, 0)), mode='constant', constant_values=-1)
            # If padded_points has more rows than num_points, truncate it
            numpy_array[j, :min(num_points, padded_points.shape[0]), :] = padded_points[:num_points]
    return numpy_array


def reconstruct_voxel_grid(seg_poly_array, grid_size=(128, 128)):
    seg_poly_array = convert_to_numpy_array(seg_poly_array, num_points=15)
    num_slices = seg_poly_array.shape[0]
    voxel_grid = np.zeros((num_slices, *grid_size), dtype=np.uint8)
    for slice_index in range(num_slices):
        points = seg_poly_array[slice_index]
        valid_points = points[points[:, 0] != -1]  # Ignore the -1 entries
        if len(valid_points) > 0:
            rr, cc = polygon(valid_points[:, 0], valid_points[:, 1], grid_size)
            voxel_grid[slice_index, rr, cc] = 1
    return voxel_grid

def create_segmentation_masks(model_outputs, ground_truths, size=(128, 128)):

    model_masks = reconstruct_voxel_grid(model_outputs)
    ground_truth_masks = reconstruct_voxel_grid(ground_truths)

    return np.array(model_masks), np.array(ground_truth_masks)



def plot_sparse_voxels(patient_num, gt_mask, pred_mask, folder, gt_color="blue", pred_color="red", view=(30, 30)):

    def pad_or_crop(arr, target_depth=95):
        # If the array is empty (all zeros), return a zero array of shape (95, 128, 128)
        if np.count_nonzero(arr) == 0:
            return np.zeros((target_depth, 128, 128), dtype=bool)
        d, h, w = arr.shape
        if d < target_depth:
            pad = (target_depth - d) // 2
            arr = np.pad(arr, ((pad, target_depth - d - pad), (0, 0), (0, 0)), mode='constant')
        elif d > target_depth:
            arr = arr[:target_depth]

        return arr.astype(bool)  # Convert to boolean for efficiency

    gt_mask = pad_or_crop(gt_mask, 95)
    pred_mask = pad_or_crop(pred_mask, 95)

    print(f"display for patient #{patient_num}")
    pred_indices = np.where(pred_mask == 1)
    gt_indices = np.where(gt_mask == 1)

    # Ensure pred_indices does not exceed gt_indices by more than 50 points
    number_excessive = 50
    if len(pred_indices[0]) > len(gt_indices[0]) + number_excessive:
        excess_points = len(pred_indices[0]) - (len(gt_indices[0]) + number_excessive)
        remove_indices = np.random.choice(len(pred_indices[0]), excess_points, replace=False)
        pred_indices = (np.delete(pred_indices[0], remove_indices),
                        np.delete(pred_indices[1], remove_indices),
                        np.delete(pred_indices[2], remove_indices))

    fig = plt.figure(figsize=(10, 5.4))
    # Ground Truth
    ax1 = fig.add_subplot(1, 2, 1, projection='3d')
    if gt_indices[0].size > 0:
        ax1.scatter(gt_indices[0], gt_indices[1], gt_indices[2], color=(30/255, 0/255, 230/255), s=1, alpha=1)
    ax1.set_xlabel('Axial Slices')
    # ax1.set_ylabel('Saggital Slices')
    # ax1.set_zlabel('Axial Slices')
    ax1.set_title(f'Test Patient ID {patient_num} - Ground Truth')
    ax1.set_xlim([0, gt_mask.shape[0]])
    ax1.set_ylim([0, gt_mask.shape[1]])
    ax1.set_zlim([0, gt_mask.shape[2]])
    ax1.view_init(elev=view[0], azim=view[1])

    # Prediction
    ax2 = fig.add_subplot(1, 2, 2, projection='3d')
    if pred_indices[0].size > 0:
        ax2.scatter(pred_indices[0], pred_indices[1], pred_indices[2], color=(128/255, 0/255, 0/255), s=1, alpha=1)
    ax2.set_xlabel('Axial Slices')
    # ax2.set_ylabel('Saggital Slices')
    # ax2.set_zlabel('Axial Slices')
    ax2.set_title(f'Test Patient ID {patient_num} - LLM Model Prediction')
    ax2.set_xlim([0, pred_mask.shape[0]])
    ax2.set_ylim([0, pred_mask.shape[1]])
    ax2.set_zlim([0, pred_mask.shape[2]])
    ax2.view_init(elev=view[0], azim=view[1])

    plt.tight_layout()
    # save the figure in a folder path
    plt.savefig(os.path.join(folder, f'patient_{patient_num}.png'))
    plt.close()
    # plt.show()




def compute_metrics(model_mask, ground_truth_mask):

    # Ensure masks are boolean
    model_mask = model_mask.astype(bool)
    ground_truth_mask = ground_truth_mask.astype(bool)

    # Compute true positives, false positives, false negatives, and true negatives
    tp = np.sum(model_mask & ground_truth_mask)  # True Positive (correctly predicted tumor)
    fp = np.sum(model_mask & ~ground_truth_mask) # False Positive (wrongly predicted tumor)
    fn = np.sum(~model_mask & ground_truth_mask) # False Negative (missed tumor)
    tn = np.sum(~model_mask & ~ground_truth_mask) # True Negative (correctly predicted background)

    # Dice Coefficient
    dice = (2 * tp) / (2 * tp + fp + fn) if (2 * tp + fp + fn) > 0 else 0.0

    # Hausdorff Distance (95%)
    if np.any(model_mask) and np.any(ground_truth_mask):  # Only compute if both have points
        model_points = np.argwhere(model_mask)
        gt_points = np.argwhere(ground_truth_mask)

        hausdorff_dist_95 = max(
            np.percentile([directed_hausdorff(model_points, gt_points)[0]], 95),
            np.percentile([directed_hausdorff(gt_points, model_points)[0]], 95)
        )
    else:
        hausdorff_dist_95 = 128  # No valid Hausdorff distance if one mask is empty

    # Precision, Recall, Specificity
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0

    return {
        "Dice Coefficient": dice,
        "95% Hausdorff Distance": hausdorff_dist_95,
        "Precision": precision,
        "Recall": recall,
        "Specificity": specificity
    }

import os
import numpy as np
import matplotlib.pyplot as plt

def list_sorted_txt_files(folder_path, images_folder, output_folder, title, START_NUM=-1):
    txt_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]
    sorted_files = sorted(txt_files, key=lambda x: int(x.split('_')[-1].split('.')[0]))

    # Store metrics
    patient_ids = []
    dice_scores = []
    hausdorff_distances = []
    precisions = []
    recalls = []
    specificities = []

    for file in sorted_files:
        patient_num = int(file.split('_')[-1].split('.')[0])
        if patient_num < START_NUM:
            continue

        test_folder = os.path.join(folder_path, file)
        model_outputs, ground_truths = parse_coordinates(test_folder)
        model_masks, ground_truth_masks = create_segmentation_masks(model_outputs, ground_truths)
        metrics = compute_metrics(model_masks, ground_truth_masks)
        print(f"Patient {patient_num}: {metrics}")

        plot_sparse_voxels(patient_num, ground_truth_masks, model_masks, images_folder, gt_color="blue", pred_color="red", view=(30, 30))

        # Store values
        patient_ids.append(patient_num)
        dice_scores.append(metrics["Dice Coefficient"])
        hausdorff_distances.append(metrics["95% Hausdorff Distance"])
        precisions.append(metrics["Precision"])
        recalls.append(metrics["Recall"])
        specificities.append(metrics["Specificity"])

    # Convert lists to numpy arrays
    patient_ids = np.array(patient_ids)
    dice_scores = np.array(dice_scores)
    hausdorff_distances = np.array(hausdorff_distances)
    precisions = np.array(precisions)
    recalls = np.array(recalls)
    specificities = np.array(specificities)


    avg_dice = np.mean(dice_scores)
    avg_hausdorff = np.mean(hausdorff_distances)
    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)
    avg_specificity = np.mean(specificities)

    print("\n===== Averages =====")
    print(f"Average Dice Coefficient: {avg_dice:.4f}")
    print(f"Average 95% Hausdorff Distance: {avg_hausdorff:.4f}")
    print(f"Average Precision: {avg_precision:.4f}")
    print(f"Average Recall: {avg_recall:.4f}")
    print(f"Average Specificity: {avg_specificity:.4f}")
    print("====================\n")

    # display and save to folders by calling function plot_metrics_distribution_box
    plot_metrics_distribution_box(patient_ids, dice_scores, hausdorff_distances, precisions, recalls, specificities, output_folder, title)
    




def plot_metrics_distribution_box(patient_ids, dice_scores, hausdorff_distances, precisions, recalls, specificities, output_folder, title):
    # Prepare the data for box plot
    all_metrics = np.array([dice_scores, hausdorff_distances, precisions, recalls, specificities]).T  # Combine metrics in rows

    # Handle any None or NaN values by replacing them with NaN
    all_metrics = np.where(all_metrics == None, np.nan, all_metrics)

    # Convert the metrics array to a DataFrame for easier manipulation
    df = pd.DataFrame(all_metrics, columns=['Dice Coefficient', '95% Hausdorff Distance', 'Precision', 'Recall', 'Specificity'])
    df = df.apply(pd.to_numeric, errors='coerce')

    # Scale Hausdorff Distance by 128 to normalize it
    df['95% Hausdorff Distance'] = df['95% Hausdorff Distance'] / 128

    # Prepare the data dictionary for boxplot
    data = {
        "Dice Coefficient": df['Dice Coefficient'].dropna(),
        "Normalised 95% Hausdorff Distance": df['95% Hausdorff Distance'].dropna(),
        "Precision": df['Precision'].dropna(),
        "Recall": df['Recall'].dropna(),
        "Specificity": df['Specificity'].dropna(),
    }

    # Convert the data dictionary to a list for the boxplot
    box_data = [data[key] for key in data]

    # Boxplot settings
    plt.figure(figsize=(10, 5))
    box = plt.boxplot(box_data, patch_artist=True, showfliers=False, tick_labels=data.keys())

    # Set custom colors with transparency
    colors = ['#90EE90', '#800080', '#1E90FF', '#4682B4', '#5F9EA0']
    for patch, color in zip(box['boxes'], colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.5)  # Make boxplot bars translucent

    # Scatter plot overlay (for better visualization)
    for i, (key, color) in enumerate(zip(data.keys(), colors)):
        y_values = data[key]
        x_values = np.random.normal(i + 1, 0.05, size=len(y_values))  # Jitter for better visualization
        plt.scatter(x_values, y_values, alpha=0.8, color=color, s=10)  # Use matching color

    # Labels and aesthetics
    plt.xlabel("Evaluation Metric Categories")
    plt.ylabel("Evaluation Metric Values")
    plt.title(title)
    plt.ylim(-0.05, 1.05)
    plt.axhline(y=0, color='black', linestyle='--', linewidth=0.8)
    plt.axhline(y=1, color='black', linestyle='--', linewidth=0.8)

    plt.grid(True)
    plt.tight_layout()  # Adjust layout to prevent overlaps

    # Save the figure to the specified folder
    save_path = f"{output_folder}/{title.replace(' ', '_')}.png"
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    
    # Optionally, you can display the plot using plt.show()
    # plt.show()
    plt.close()



In [22]:
folder_path = "/Users/felicialiu/Desktop/ESC499/Code2/segpoly_noref/segpoly_noref_general_results"
images_folder = "/Users/felicialiu/Desktop/ESC499/Code2/segpoly_noref/gen_images"
output_folder = "/Users/felicialiu/Desktop/ESC499/Code2/segpoly_noref/images"
title = "General LLM Bounding Polygon Segmentation Results on Axial Slices"
list_sorted_txt_files(folder_path, images_folder, output_folder, title)

Patient 0: {'Dice Coefficient': 0.10050935740390796, '95% Hausdorff Distance': 88.09086218218096, 'Precision': 0.11806197183098592, 'Recall': 0.08750041754350803, 'Specificity': 0.9448923784875256}
display for patient #0
Patient 1: {'Dice Coefficient': 0.03982930298719772, '95% Hausdorff Distance': 98.80283396745257, 'Precision': 0.023400576368876082, 'Recall': 0.13368455712874547, 'Specificity': 0.9301920295975248}
display for patient #1
Patient 2: {'Dice Coefficient': 0.0, '95% Hausdorff Distance': 128, 'Precision': 0.0, 'Recall': 0.0, 'Specificity': 0.0}
display for patient #2
Patient 3: {'Dice Coefficient': 0.02752216690970651, '95% Hausdorff Distance': 143.7358688706476, 'Precision': 0.018897456554264013, 'Recall': 0.05062900838677849, 'Specificity': 0.9365631213611306}
display for patient #3
Patient 4: {'Dice Coefficient': 0.000552791597567717, '95% Hausdorff Distance': 113.58697108383514, 'Precision': 0.00029478959392733434, 'Recall': 0.004429678848283499, 'Specificity': 0.94807

In [23]:
folder_path = "/Users/felicialiu/Desktop/ESC499/Code2/segpoly_noref/segpoly_noref_100steps_results"
images_folder = "/Users/felicialiu/Desktop/ESC499/Code2/segpoly_noref/finetune_images"
output_folder = "/Users/felicialiu/Desktop/ESC499/Code2/segpoly_noref/images"
title = "Fine-Tuned LLM Bounding Polygon Segmentation Results on Axial Slices"
list_sorted_txt_files(folder_path, images_folder, output_folder, title)

Patient 0: {'Dice Coefficient': 0.04134341182770258, '95% Hausdorff Distance': 95.03683496413377, 'Precision': 0.08125703656566645, 'Recall': 0.027724888933426863, 'Specificity': 0.9735712656334926}
display for patient #0
Patient 1: {'Dice Coefficient': 0.0, '95% Hausdorff Distance': 91.43850392476902, 'Precision': 0.0, 'Recall': 0.0, 'Specificity': 0.9807270015614508}
display for patient #1
Patient 2: {'Dice Coefficient': 0.0, '95% Hausdorff Distance': 128, 'Precision': 0.0, 'Recall': 0.0, 'Specificity': 0.0}
display for patient #2
Patient 3: {'Dice Coefficient': 0.006078489229861316, '95% Hausdorff Distance': 124.36237373096414, 'Precision': 0.005827769605069593, 'Recall': 0.006351751356684756, 'Specificity': 0.9738492540689853}
display for patient #3
Patient 4: {'Dice Coefficient': 0.0, '95% Hausdorff Distance': 119.30213744941874, 'Precision': 0.0, 'Recall': 0.0, 'Specificity': 0.980248123380327}
display for patient #4
Patient 5: {'Dice Coefficient': 0.09211911100105244, '95% Hausd